In [1]:
!pip install pandas numpy sqlalchemy openpyxl


In [1]:
import pandas as pd

df = pd.read_excel(r"D:\NXT_Wave tasks\siddharth_trade_pipeline\data\raw\Sample_Data.xlsx")
df.head()


,PORT CODE,DATE,IEC,HS CODE,GOODS DESCRIPTION,Master category,Model Name,Model Number,Capacity,Qty,Unit of measure,Price,Unit of measure.1,QUANTITY,UNIT,UNIT PRICE_INR,TOTAL VALUE_INR,UNIT PRICE_USD,TOTAL VALUE_USD,DUTY PAID_INR
0,INNSA1,2025-10-28,301049751,73239990,TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS/USD 2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,290.0,KGS,401.725310,116500.34,4.529034,1313.419842,32736.6
1,INNSA1,2025-10-28,301049751,73231000,8001-2 STEEL SCRUBBER 2PCS SET (QTY: 336000 SE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9800.0,KGS,440.602218,4317901.74,4.967331,48679.839230,1898149.5
2,INNSA1,2025-10-23,301049751,73239990,MILD STEEL MULTI FUNCTION CLOTH STAND (RYX-02-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4004.0,KGS,408.893664,1637210.23,4.609850,18457.837993,460056.0
3,INNSA1,2025-10-13,301049751,73239990,SB-12 STEEL TEA STRAINER BIG (QTY 6336 PCS/USD...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1548.8,KGS,378.457845,586155.51,4.219151,6534.621070,164709.7
4,INNSA1,2025-10-13,301049751,73239990,SB-12 STEEL TEA STRAINER BIG (QTY 43038 PCS/US...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10042.2,KGS,399.739675,4014265.76,4.456407,44752.126644,1128008.7


In [2]:
df.columns


Index(['PORT CODE', 'DATE', 'IEC', 'HS CODE', 'GOODS DESCRIPTION',
       'Master category', 'Model Name', 'Model Number', 'Capacity', 'Qty',
       'Unit of measure', 'Price', 'Unit of measure.1', 'QUANTITY', 'UNIT',
       'UNIT PRICE_INR', 'TOTAL VALUE_INR', 'UNIT PRICE_USD',
       'TOTAL VALUE_USD', 'DUTY PAID_INR'],
      dtype='object')

Convert DATE to Actual Date

In [3]:
df["date_of_shipment"] = pd.to_datetime(df["DATE"], errors="coerce")
df["year"] = df["date_of_shipment"].dt.year
df["month"] = df["date_of_shipment"].dt.month

df.head()


,PORT CODE,DATE,IEC,HS CODE,GOODS DESCRIPTION,Master category,Model Name,Model Number,Capacity,Qty,...,QUANTITY,UNIT,UNIT PRICE_INR,TOTAL VALUE_INR,UNIT PRICE_USD,TOTAL VALUE_USD,DUTY PAID_INR,date_of_shipment,year,month
0,INNSA1,2025-10-28,301049751,73239990,TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS/USD 2...,NaN,NaN,NaN,NaN,NaN,...,290.0,KGS,401.725310,116500.34,4.529034,1313.419842,32736.6,2025-10-28,2025,10
1,INNSA1,2025-10-28,301049751,73231000,8001-2 STEEL SCRUBBER 2PCS SET (QTY: 336000 SE...,NaN,NaN,NaN,NaN,NaN,...,9800.0,KGS,440.602218,4317901.74,4.967331,48679.839230,1898149.5,2025-10-28,2025,10
2,INNSA1,2025-10-23,301049751,73239990,MILD STEEL MULTI FUNCTION CLOTH STAND (RYX-02-...,NaN,NaN,NaN,NaN,NaN,...,4004.0,KGS,408.893664,1637210.23,4.609850,18457.837993,460056.0,2025-10-23,2025,10
3,INNSA1,2025-10-13,301049751,73239990,SB-12 STEEL TEA STRAINER BIG (QTY 6336 PCS/USD...,NaN,NaN,NaN,NaN,NaN,...,1548.8,KGS,378.457845,586155.51,4.219151,6534.621070,164709.7,2025-10-13,2025,10
4,INNSA1,2025-10-13,301049751,73239990,SB-12 STEEL TEA STRAINER BIG (QTY 43038 PCS/US...,NaN,NaN,NaN,NaN,NaN,...,10042.2,KGS,399.739675,4014265.76,4.456407,44752.126644,1128008.7,2025-10-13,2025,10


Extract Unit Price (USD) From Description

In [5]:
import re

def extract_unit_price_usd(desc):
    if not isinstance(desc, str):
        return None
    match = re.search(r"USD\s*([\d\.]+)", desc.upper())
    return float(match.group(1)) if match else None

df["unit_price_usd_parsed"] = df["GOODS DESCRIPTION"].apply(extract_unit_price_usd)


Extract Embedded Quantity

In [6]:
def extract_embedded_qty(desc):
    if not isinstance(desc, str):
        return None
    match = re.search(r"QTY[:\s]*([0-9]+)", desc.upper())
    return int(match.group(1)) if match else None

df["embedded_quantity"] = df["GOODS DESCRIPTION"].apply(extract_embedded_qty)


Extract Model Number
Data has codes like TH5170, 8001-2, AM-967, etc

In [7]:
def extract_model(desc):
    if not isinstance(desc, str):
        return None
    match = re.search(r"\b([A-Z0-9]+(?:-[A-Z0-9]+)*)\b", desc)
    return match.group(1) if match else None

df["model_number"] = df["GOODS DESCRIPTION"].apply(extract_model)


Standardize Units (PCS, KG, MT, etc.)

In [8]:
unit_map = {
    "PCS": "PCS", "PC": "PCS", "NOS": "PCS", "PIECES": "PCS",
    "KG": "KG", "KGS": "KG",
    "MT": "MT"
}

def normalize_unit(u):
    if not isinstance(u, str):
        return None
    return unit_map.get(u.strip().upper(), u.strip().upper())

df["unit_standardized"] = df["UNIT"].apply(normalize_unit)


Calculate Grand Total & Landed Cost

In [ ]:
df["grand_total_inr"] = df["TOTAL VALUE_INR"] + df["DUTY PAID_INR"]

def landed_cost(row):
    qty = row["QUANTITY"]
    if qty and qty > 0:
        return row["grand_total_inr"] / qty
    return None

df["landed_cost_per_unit"] = df.apply(landed_cost, axis=1)


In [10]:
df.to_csv(r"D:\NXT_Wave tasks\siddharth_trade_pipeline\data\processed\trade_cleaned.csv", index=False)


Installing SQLAlchemy

In [11]:
!pip install sqlalchemy
